In [9]:
from tqdm.notebook import tqdm
import math
import gym
import torch
import torch.optim as optim 
from torch.utils.tensorboard import SummaryWriter
from collections import deque
import numpy as np

from networks.dqn_atari import ENS_DQN_Large
# from active_rl.utils.memory import RankedReplayMemory, LabeledReplayMemory
# from baselines.deepq.replay_buffer import PrioritizedReplayBuffer
from active_rl.utils.priority_replay import PrioritizedReplayBuffer
from active_rl.utils.optimization import standard_opt_ens_priority_td
from active_rl.environments.atari_wrappers import make_atari, wrap_deepmind
from active_rl.utils.atari_utils import fp, ActionSelector, evaluate
from active_rl.utils.acquisition_functions import ens_BALD

In [2]:
env_name = 'Breakout'
env_raw = make_atari('{}NoFrameskip-v4'.format(env_name))
env = wrap_deepmind(env_raw, frame_stack=False, episode_life=True, clip_rewards=True)
c,h,w = c,h,w = fp(env.reset()).shape
n_actions = env.action_space.n

In [3]:
BATCH_SIZE = 64
LR = 0.0000625
GAMMA = 0.99
EPS_START = 1.
EPS_END = 0.1
EPS_DECAY = 1000000 
NUM_STEPS = 20000000
POLICY_UPDATE = 4
TARGET_UPDATE = 4000
EVALUATE_FREQ = 10000
MEMORY_SIZE = 100000
PRIORITY_ALPHA=0.7
INITIAL_STEPS=1000
PERCENTAGE = 0.5

NAME = 'standard_ens_large_TD_priority_test'

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # if gpu is to be used
policy_net = ENS_DQN_Large(n_actions).to(device)
target_net = ENS_DQN_Large(n_actions).to(device)
policy_net.apply(policy_net.init_weights)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.Adam(policy_net.parameters(), lr=LR, eps=1.5e-4)

In [5]:
# memory = RankedReplayMemory(M_SIZE, [5,h,w], n_actions, mc_BALD_ens, AMN_net, replacement=False, device=device)
memory = PrioritizedReplayBuffer(MEMORY_SIZE, PRIORITY_ALPHA)
action_selector = ActionSelector(EPS_START, EPS_END, policy_net, EPS_DECAY, n_actions, device)

In [6]:
steps_done = 0
writer = SummaryWriter(f'runs/{NAME}')

In [7]:
q = deque(maxlen=5)
done=True
eps = 0
episode_len = 0

In [10]:
progressive = tqdm(range(NUM_STEPS), total=NUM_STEPS, ncols=400, leave=False, unit='b')
for step in progressive:
  if done:
    env.reset()
    sum_reward = 0
    episode_len = 0
    img, _, _, _ = env.step(1) # BREAKOUT specific !!!
    for i in range(10): # no-op
      n_frame, _, _, _ = env.step(0)
      n_frame = fp(n_frame)
      q.append(n_frame)
        
  # Select and perform an action
  state = torch.cat(list(q))[1:].unsqueeze(0)
  action, eps = action_selector.select_action(state)
  n_frame, reward, done, info = env.step(action)
  n_frame = fp(n_frame)

  # 5 frame as memory
  q.append(n_frame)
  cur_state = torch.cat(list(q))[:-1].numpy()
  next_state = torch.cat(list(q))[1:].numpy()
  
  memory.add(cur_state, action, reward, next_state, done) 
  episode_len += 1
    
  if step % POLICY_UPDATE == 0 and step > INITIAL_STEPS:
    loss = standard_opt_ens_priority_td(policy_net, target_net, optimizer, memory, 
                                        batch_size=BATCH_SIZE, device=device)
    if loss is not None:
      writer.add_scalar('Performance/loss', loss, step)
      
  if step % TARGET_UPDATE == 0 and step > INITIAL_STEPS:
    target_net.load_state_dict(policy_net.state_dict())
  
  if step % EVALUATE_FREQ == 0 and step > INITIAL_STEPS:
    evaluated_reward = evaluate(step, policy_net, device, env_raw, n_actions, eps=0.05, num_episode=15)
    writer.add_scalar('Performance/reward', evaluated_reward, step)

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 7.80 GiB total capacity; 779.18 MiB already allocated; 25.00 MiB free; 786.00 MiB reserved in total by PyTorch)